In [1]:
# 🚀 Delhi Traffic ML App: Dynamic Least Congested Route Finder

# 📦 Install osmnx if not already installed
try:
    import osmnx as ox
except ImportError:
    !pip install osmnx
    import osmnx as ox

import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import folium
import webbrowser

# 📂 Path to your speed dataset
speed_csv_path = r"C:\Users\sagni\Downloads\Road Traffic\archive (1)\new_delhi_traffic_dataset\weekday_stats\2024_week_day_speed_city.csv"

# 📥 Load speed dataset
speed_df = pd.read_csv(speed_csv_path)
print("✅ Speed dataset loaded. Shape:", speed_df.shape)

# 🧹 Clean speed dataset: strip " km/h" and convert to float
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for day in days:
    speed_df[day] = speed_df[day].str.replace(' km/h', '', regex=False).astype(float)

# 📅 User selects day
day_of_week = 'Monday'   # Change as needed
mean_city_speed = speed_df[day_of_week].mean()
print(f"📊 Average speed for {day_of_week}: {mean_city_speed:.2f} km/h")

# 🌐 Download full road network of Delhi from OpenStreetMap
print("🌐 Downloading Delhi road network from OSM...")
G = ox.graph_from_place("Delhi, India", network_type='drive')
print("✅ Delhi road network loaded. Nodes:", len(G.nodes), "Edges:", len(G.edges))

# 🏎️ Add speed limits to edges from speed_df (simulate mapping)
for u, v, k, data in G.edges(keys=True, data=True):
    data['speed_kmh'] = mean_city_speed
    length_m = data.get('length', 100)  # fallback 100m if no length
    speed_ms = data['speed_kmh'] * 1000 / 3600
    data['travel_time_sec'] = length_m / speed_ms if speed_ms > 0 else 9999

print("✅ Speed data applied to OSM network edges.")

# 📍 Input origin and destination
start_location = input("Enter Start Location (e.g., India Gate, Delhi): ")
end_location = input("Enter End Location (e.g., Jama Masjid, Delhi): ")

# 🗺️ Geocode locations to lat/lon
orig_point = ox.geocoder.geocode(start_location)
dest_point = ox.geocoder.geocode(end_location)

print(f"📌 Start: {start_location} at {orig_point}")
print(f"📌 End: {end_location} at {dest_point}")

# 🔎 Find nearest nodes in graph
orig_node = ox.distance.nearest_nodes(G, orig_point[1], orig_point[0])
dest_node = ox.distance.nearest_nodes(G, dest_point[1], dest_point[0])

# 🛣️ Compute least congested path (Dijkstra using travel_time_sec)
route = nx.shortest_path(G, orig_node, dest_node, weight='travel_time_sec')
print("✅ Route found. Number of steps:", len(route))

# 🌍 Visualize route with Folium
m = folium.Map(location=orig_point, zoom_start=13)
route_coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route]
folium.PolyLine(route_coords, color="blue", weight=5, tooltip="Fastest Route").add_to(m)
folium.Marker(orig_point, popup=f"Start: {start_location}", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(dest_point, popup=f"End: {end_location}", icon=folium.Icon(color="red")).add_to(m)

# 💾 Save and open map
output_map = r"C:\Users\sagni\Downloads\Road Traffic\delhi_dynamic_least_congested_route.html"
m.save(output_map)
print(f"✅ Map saved as {output_map}")

# 🌐 Automatically open in default browser
webbrowser.open(output_map)
print("🌍 Opening map in your default browser...")


✅ Speed dataset loaded. Shape: (12, 8)
📊 Average speed for Monday: 30.00 km/h
🌐 Downloading Delhi road network from OSM...
✅ Delhi road network loaded. Nodes: 183023 Edges: 498303
✅ Speed data applied to OSM network edges.


Enter Start Location (e.g., India Gate, Delhi):  India Gate
Enter End Location (e.g., Jama Masjid, Delhi):  Qutab Minar


📌 Start: India Gate at (28.6129332, 77.2294928)
📌 End: Qutab Minar at (28.512951, 77.1864532)
✅ Route found. Number of steps: 100
✅ Map saved as C:\Users\sagni\Downloads\Road Traffic\delhi_dynamic_least_congested_route.html
🌍 Opening map in your default browser...
